# LifeSnaps Feature Selection Experiments v2

Expermiments amb un menor nombre de features.

#### Import de llibreries necessaries

In [12]:
import warnings
warnings.filterwarnings('ignore')

# Data handling and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Imbalanced data pipeline
from imblearn.pipeline import Pipeline as ImbPipeline

# Core utilities
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.inspection import permutation_importance

# Model definitions fora dels defints
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


############## IMPORTS DEL NOSTRE PAQUET ###################
# Obtenim les funcions per entrenar i evaluar els models i registrar les mètriques
from ai_health_assistant.utils.train_helpers import train_models, append_results, plot_learning_curve, mat_confusio, update_experiments_file

# Obtenim els classificadors i els seus parametres
from ai_health_assistant.utils.model_config import get_classifier_config, PARAM_GRIDS, CLASSIFIERS, BALANCING_METHODS

# Obtenim el target, features el la construcció del preprocessador
from ai_health_assistant.utils.prep_helpers import TARGET, build_preprocessor, FEATURES

# Configuració de pandas
pd.set_option('display.max_columns', None)

# Carrega de dades, netes i amb fe aplicat
df_train = pd.read_csv('../data/df_engineered_train.csv')
df_test = pd.read_csv('../data/df_engineered_test.csv')
    
print(f"Shape: {df_train.shape}")
print(f"Shape: {df_test.shape}")


Shape: (1832, 40)
Shape: (458, 40)


## Lectura de dades i split de train / test

### Train / Test Split

In [13]:
# Fem l'split de les dades, separant les features i el target
X_train = df_train[FEATURES]
y_train = df_train[TARGET]

X_test = df_test[FEATURES]
y_test = df_test[TARGET]

print(f"\nTrain shape: {X_train.shape}")
print(f"Test shape: {X_test.shape}")
print('\n','--'*50)
print(f"\nDistribució train:\n{y_train.value_counts(normalize=True)}")
print(f"\nDistribució test:\n{y_test.value_counts(normalize=True)}")

numerical_features = X_train.select_dtypes(include=['number']).columns.tolist()
categorical_features = X_train.select_dtypes(exclude=['number']).columns.tolist()

print(f"\nCol. numeriques ({len(numerical_features)}): \n{numerical_features}")
print(f"Col. categoriques ({len(categorical_features)}): \n{categorical_features}")



Train shape: (1832, 37)
Test shape: (458, 37)

 ----------------------------------------------------------------------------------------------------

Distribució train:
TIRED
0.0    0.615721
1.0    0.384279
Name: proportion, dtype: float64

Distribució test:
TIRED
0.0    0.615721
1.0    0.384279
Name: proportion, dtype: float64

Col. numeriques (35): 
['bmi', 'calories', 'steps', 'lightly_active_minutes', 'moderately_active_minutes', 'very_active_minutes', 'sedentary_minutes', 'resting_hr', 'minutes_below_default_zone_1', 'minutes_in_default_zone_1', 'minutes_in_default_zone_2', 'minutes_in_default_zone_3', 'minutesAsleep', 'minutesAwake', 'sleep_efficiency', 'sleep_deep_ratio', 'sleep_light_ratio', 'sleep_rem_ratio', 'sleep_wake_ratio', 'daily_temperature_variation', 'rmssd', 'spo2', 'full_sleep_breathing_rate', 'wake_after_sleep_pct', 'steps_norm_cal', 'deep_sleep_score', 'active_sedentary_ratio', 'sleep_activity_balance', 'bmi_hr_interaction', 'sleep_quality_index', 'hr_zone_variab

### Definim el preprocessador

In [14]:
preprocessor = build_preprocessor(df_train, FEATURES)

## ENTRENAMENT DEL MODEL BASE

In [15]:
display_cols = ['Experiment', 'Train F1 (1)', 'Train F1 (macro global)','Train Accuracy', 'Test Recall (1)', 'Test Precision (1)', 'Test F1 (1)', 'Test F1 (macro global)','Test Accuracy']

# --------------------------------------------------------------
# Filtre de selecció dels models a entrenar
PROVA_MODELS = ["LGBM"]
# Selecció del mètode de balanceig
balance_name = "SMOTETomek"
# --------------------------------------------------------------

# Filtre per no haver de entrenar tots el models
CLASSIFIERS_FILTER = {k: v for k, v in CLASSIFIERS.items() if k in PROVA_MODELS}
balance_method = BALANCING_METHODS[balance_name]

### Regressió Logistica

In [16]:

reg_results = []
reg_models = {}

reg_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(
        max_iter=2000,            # convergència assegurada
        class_weight="balanced", # tracta l’imbalance de la classe 1
        solver="lbfgs",          # ràpid i estable per datasets petits/mitjans
    ))
])

reg_param_grid = {
    "classifier__C": [0.001, 0.01, 0.1, 1, 10, 100, 200, 500, 1000]
}

best_est, y_train_pred, train_report, y_test_pred, test_report, best_params, best_score = train_models(
        X_train, 
        y_train,
        X_test,
        y_test, 
        reg_pipeline, 
        reg_param_grid,
        search_type='grid',
    )

reg_results_df = append_results(
    reg_results,
    "LogisticRegression",
    train_report,
    test_report,
    best_params,
    best_score,
    experiment="EntrenamentBasic"
)

display(reg_results_df[display_cols])
update_experiments_file(reg_results_df)

Entrenant model...

Train F1 (1): 0.5156 | Test F1 (1): 0.4800 | Train Acc: 0.5939 | Test Acc: 0.5742
              precision    recall  f1-score   support

         0.0     0.6680    0.6135    0.6396       282
         1.0     0.4523    0.5114    0.4800       176

    accuracy                         0.5742       458
   macro avg     0.5601    0.5624    0.5598       458
weighted avg     0.5851    0.5742    0.5782       458



,Experiment,Train F1 (1),Train F1 (macro global),Train Accuracy,Test Recall (1),Test Precision (1),Test F1 (1),Test F1 (macro global),Test Accuracy
0,LogisticRegression_EntrenamentBasic,0.51562,0.583,0.59389,0.51136,0.45226,0.48,0.55978,0.57424



Métriques guardades a ../results/02_experiments/experiments.csv



In [17]:
base_results = []

for model, classifier in CLASSIFIERS_FILTER.items():

    if model == "BalancedRandomForest":
        pipeline = ImbPipeline([
            ("preprocessor", preprocessor),
            ("classifier", classifier)
        ])
    else:
        pipeline = ImbPipeline([
            ("preprocessor", preprocessor),
            ("balance", balance_method),
            ("classifier", classifier)
        ])

    print(f"\n==== {model} ====")
    best_est, y_train_pred, train_report, y_test_pred, test_report, best_params, best_score = train_models(
        X_train,
        y_train,
        X_test,
        y_test, 
        pipeline,
        PARAM_GRIDS[model],
        search_type='grid'
    )

    base_results_df = append_results(
    base_results,
    model,
    train_report,
    test_report,
    best_params,
    best_score,
    experiment="EntrenamentBasic"
)


==== LGBM ====
Entrenant model...
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

In [18]:
update_experiments_file(base_results_df)


Métriques guardades a ../results/02_experiments/experiments.csv



## EXPERIMENT 1: Importancia de les caracteristiques

Entrenem RandomForest per indentificar les caracteristiques més importants (3 i 5), posteriorment entrenem els models utilitzant aquestes 3 i 5 característiques, per veure si augmenta el rendiment del model. Proavarem tambe amb permutation importances.

### Feature Importance

La Gini importance d’una feature és: La suma de totes les reduccions d’impuresa (Gini) que ha causat al llarg de tots els arbres i de totes les seves aparicions.

In [19]:
# Param grid & pipeline bàsic de random forest
rf_name = "BalancedRandomForest"

pipeline = ImbPipeline([
    ("preprocessor", preprocessor),
    ("classifier", CLASSIFIERS[rf_name])
])

# Entrenament del model
best_est, y_train_pred, train_report, y_test_pred, test_report, best_params, best_score = train_models(
    X_train, 
    y_train,
    X_test,
    y_test, 
    pipeline, 
    PARAM_GRIDS[rf_name],
    search_type='grid'
)
preprocessor = best_est.named_steps['preprocessor']

feature_names = preprocessor.get_feature_names_out()

# Serie em els noms
importances_raw = pd.Series(
    best_est.named_steps["classifier"].feature_importances_,
    index=feature_names
)

agg_importances = (
    importances_raw.groupby(feature_names).sum().sort_values(ascending=False)
)

# Top-4 a Top-15 importàncies

top3 = agg_importances.head(3).index.tolist()
top4 = agg_importances.head(4).index.tolist() 
top5 = agg_importances.head(5).index.tolist()
top6 = agg_importances.head(6).index.tolist()
top7 = agg_importances.head(7).index.tolist()
top8 = agg_importances.head(8).index.tolist()
top9 = agg_importances.head(9).index.tolist()
top10 = agg_importances.head(10).index.tolist()
top11 = agg_importances.head(11).index.tolist()
top12 = agg_importances.head(12).index.tolist()
top13 = agg_importances.head(13).index.tolist()
top14 = agg_importances.head(14).index.tolist()
top15 = agg_importances.head(15).index.tolist() #FEM FINS A 20
top16 = agg_importances.head(16).index.tolist()
top17 = agg_importances.head(17).index.tolist()
top18 = agg_importances.head(18).index.tolist()
top19 = agg_importances.head(19).index.tolist()
top20 = agg_importances.head(20).index.tolist()

print("Top-3 features:", top3)
print("Top-4 features:", top4)
print("Top-5 features:", top5)
print("Top-6 features:", top6)
print("Top-7 features:", top7)
print("Top-8 features:", top8)
print("Top-9 features:", top9)
print("Top-10 features:", top10)
print("Top-11 features:", top11)
print("Top-12 features:", top12)
print("Top-13 features:", top13)
print("Top-14 features:", top14)
print("Top-15 features:", top15)
print("Top-16 features:", top16)
print("Top-17 features:", top17)
print("Top-18 features:", top18)
print("Top-19 features:", top19)
print("Top-20 features:", top20)



Entrenant model...

Train F1 (1): 0.7136 | Test F1 (1): 0.5793 | Train Acc: 0.6960 | Test Acc: 0.5306
              precision    recall  f1-score   support

         0.0     0.7724    0.3369    0.4691       282
         1.0     0.4418    0.8409    0.5793       176

    accuracy                         0.5306       458
   macro avg     0.6071    0.5889    0.5242       458
weighted avg     0.6453    0.5306    0.5115       458

Top-3 features: ['num__calories', 'num__bmi_hr_interaction', 'num__bmi']
Top-4 features: ['num__calories', 'num__bmi_hr_interaction', 'num__bmi', 'num__resting_hr']
Top-5 features: ['num__calories', 'num__bmi_hr_interaction', 'num__bmi', 'num__resting_hr', 'num__steps_norm_cal']
Top-6 features: ['num__calories', 'num__bmi_hr_interaction', 'num__bmi', 'num__resting_hr', 'num__steps_norm_cal', 'num__daily_temperature_variation']
Top-7 features: ['num__calories', 'num__bmi_hr_interaction', 'num__bmi', 'num__resting_hr', 'num__steps_norm_cal', 'num__daily_temperature_v

### Rentrenament de models 4-15 millors features

In [20]:
# Definim els conjunts de features
feature_sets = {
    "Top3": top3,
    "Top4": top4,   
    "Top5": top5,
    "Top6": top6,
    "Top7": top7,
    "Top8": top8,
    "Top9": top9,
    "Top10": top10,
    "Top11": top11,
    "Top12": top12,
    "Top13": top13,
    "Top14": top14,
    "Top15": top15,
    "Top16": top16,
    "Top17": top17,
    "Top18": top18,
    "Top19": top19,
    "Top20": top20,
}

# Reentrenament i avaluació per a cada subset
fi_results = []
preprocessor = build_preprocessor(X_train, FEATURES)
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)
feature_names = preprocessor.get_feature_names_out()

# Generem el train transformat i el test transformat
X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names, index=X_train.index)
X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names, index=X_test.index)


for label, feats in feature_sets.items():
    print(f"\nEntrenament amb {label}")
    for model, classifier in CLASSIFIERS_FILTER.items():

        print(f'==== {model} ====')
        if model == "BalancedRandomForest": # Em dona millors resutats sense balance
            pipe = ImbPipeline([
                ("classifier", classifier)
            ])
        else:
            pipe = ImbPipeline([
                ("balance", balance_method),
                ("classifier", classifier)
            ])

        best_est, y_train_pred, train_report, y_test_pred, test_report, best_params, best_score = train_models(
            X_train_transformed[feats],
            y_train,
            X_test_transformed[feats],
            y_test,
            pipe,
            PARAM_GRIDS[model]
        )

        fi_results_df =append_results(
            fi_results,
            model,
            train_report,
            test_report,
            best_params,
            best_score,
            experiment=f"FI_{label}"
        )


Entrenament amb Top3
==== LGBM ====
Entrenant model...
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with 

In [21]:
display(fi_results_df[display_cols])
update_experiments_file(fi_results_df)

,Experiment,Train F1 (1),Train F1 (macro global),Train Accuracy,Test Recall (1),Test Precision (1),Test F1 (1),Test F1 (macro global),Test Accuracy
0,LGBM_FI_Top3,0.61945,0.60656,0.60699,0.70455,0.42177,0.52766,0.51495,0.51528
1,LGBM_FI_Top4,0.64637,0.63847,0.63865,0.71591,0.43750,0.54310,0.53704,0.53712
2,LGBM_FI_Top5,0.66988,0.66307,0.66321,0.67045,0.43704,0.52915,0.54117,0.54148
3,LGBM_FI_Top6,0.67466,0.67413,0.67413,0.77841,0.48754,0.59956,0.60043,0.60044
4,LGBM_FI_Top7,0.70303,0.70576,0.70579,0.71023,0.44170,0.54466,0.54367,0.54367
5,LGBM_FI_Top8,0.69934,0.70194,0.70197,0.72159,0.45357,0.55702,0.55894,0.55895
6,LGBM_FI_Top9,0.71973,0.73047,0.73090,0.70455,0.45421,0.55234,0.56097,0.56114
7,LGBM_FI_Top10,0.72675,0.73955,0.74017,0.69886,0.46067,0.55530,0.56941,0.56987
8,LGBM_FI_Top11,0.73946,0.75307,0.75382,0.70455,0.45091,0.54989,0.55666,0.55677
9,LGBM_FI_Top12,0.73781,0.74996,0.75055,0.72159,0.46520,0.56570,0.57407,0.57424



Métriques guardades a ../results/02_experiments/experiments.csv



### Permutation Importance

Per avaluar la importància de les característiques del model. Serveix per determinar quines característiques tenen més impacte en el rendiment del model.

In [22]:
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt

# Train el random forest
rf_name = 'BalancedRandomForest'
clf_rf, param_grid_rf = get_classifier_config(rf_name)

# nou pipeline
pipe = ImbPipeline([
    ("preprocessor", preprocessor),
    ("classifier", clf_rf)
])

# Train el model
best_est, y_train_pred, train_report, y_test_pred, test_report, best_params, best_score = train_models( 
    X_train, 
    y_train,
    X_test,
    y_test, 
    pipe, 
    param_grid_rf,
    search_type='grid'
)

best_rf_model = best_est

preprocessor = best_rf_model.named_steps['preprocessor']
classifier = best_rf_model.named_steps['classifier']

X_test_transformed = preprocessor.transform(X_test)

feature_names = preprocessor.get_feature_names_out()



# pipeline nomes classifier
final_estimator = Pipeline([
    ('classifier', classifier)
])

# Calculem la importància permutativa
result = permutation_importance(
    final_estimator,
    X_test_transformed,  # Conjunt de dades que no hagi vist
    y_test,
    n_repeats=200,
    n_jobs=-1,
    random_state=42
)

# Creem Series amb els noms de les caracteristiques
perm_importances = pd.Series(
    result.importances_mean, 
    index=feature_names
).sort_values(ascending=False)

# Top 3 a 15 features
perm_top_features = {
    3: perm_importances.head(3).index.tolist(),
    4: perm_importances.head(4).index.tolist(),
    5: perm_importances.head(5).index.tolist(),
    6: perm_importances.head(6).index.tolist(),
    7: perm_importances.head(7).index.tolist(),
    8: perm_importances.head(8).index.tolist(),
    9: perm_importances.head(9).index.tolist(),
    10: perm_importances.head(10).index.tolist(),
    11: perm_importances.head(11).index.tolist(),
    12: perm_importances.head(12).index.tolist(),
    13: perm_importances.head(13).index.tolist(),
    14: perm_importances.head(14).index.tolist(),
    15: perm_importances.head(15).index.tolist(),
    16: perm_importances.head(16).index.tolist(),
    17: perm_importances.head(17).index.tolist(),
    18: perm_importances.head(18).index.tolist(),
    19: perm_importances.head(19).index.tolist(),
    20: perm_importances.head(20).index.tolist(),
}

print("\nTop-15 features (Permutation):")
display(perm_importances.head(20).to_frame("Importancia"))

Entrenant model...

Train F1 (1): 0.7136 | Test F1 (1): 0.5793 | Train Acc: 0.6960 | Test Acc: 0.5306
              precision    recall  f1-score   support

         0.0     0.7724    0.3369    0.4691       282
         1.0     0.4418    0.8409    0.5793       176

    accuracy                         0.5306       458
   macro avg     0.6071    0.5889    0.5242       458
weighted avg     0.6453    0.5306    0.5115       458


Top-15 features (Permutation):


,Importancia
num__bmi,0.025666
num__recovery_factor,0.020742
num__minutesAsleep,0.019367
num__full_sleep_breathing_rate,0.013919
num__daily_temperature_variation,0.012609
num__minutes_in_default_zone_1,0.011430
num__wake_after_sleep_pct,0.010993
num__calories,0.009290
num__active_to_rest_transition,0.008319
num__rmssd,0.008166


### Reentrenament Permutation importance

In [23]:
# Definim els dos conjunts de features basats en importància de permutació
perm_sets = {
    "Perm_Top3": perm_top_features[3],
    "Perm_Top4": perm_top_features[4],
    "Perm_Top5": perm_top_features[5], 
    "Perm_Top6": perm_top_features[6],
    "Perm_Top7": perm_top_features[7],
    "Perm_Top8": perm_top_features[8],
    "Perm_Top9": perm_top_features[9],
    "Perm_Top10": perm_top_features[10],
    "Perm_Top11": perm_top_features[11],
    "Perm_Top12": perm_top_features[12],
    "Perm_Top13": perm_top_features[13],
    "Perm_Top14": perm_top_features[14],
    "Perm_Top15": perm_top_features[15],
    "Perm_Top16": perm_top_features[16],
    "Perm_Top17": perm_top_features[17],
    "Perm_Top18": perm_top_features[18],
    "Perm_Top19": perm_top_features[19],
    "Perm_Top20": perm_top_features[20],
}

print(perm_top_features[20])

# Reentrenament i avaluació per a cada subset
perm_results = []

# Preprocessem un cop fora del bucle per eficiència
preprocessor = build_preprocessor(X_train, FEATURES)
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# Convertim a DataFrame mantenint els índexs
feature_names = preprocessor.get_feature_names_out()
X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names, index=X_train.index)
X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names, index=X_test.index)

for label, feat_indices in perm_sets.items():
    print(f"\nEntrenament models amb {label}")
    
    for model_name, classifier in CLASSIFIERS_FILTER.items():
        print(f'==== {model_name} ====')
        
        if model_name == "BalancedRandomForest":
            pipe = ImbPipeline([
                ("classifier", classifier)
            ])
        else:
            pipe = ImbPipeline([
                ("balance", balance_method),
                ("classifier", classifier)
            ])

        best_est, y_train_pred, train_report, y_test_pred, test_report, best_params, best_score = train_models(
            X_train_transformed[feat_indices],
            y_train,
            X_test_transformed[feat_indices],
            y_test,
            pipe,
            PARAM_GRIDS[model_name]
        )

        perm_results_df = append_results(
            perm_results,
            model_name,
            train_report,
            test_report,
            best_params,
            best_score,
            experiment=label  # Usem directament l'etiqueta (Perm_TopX)
        )

['num__bmi', 'num__recovery_factor', 'num__minutesAsleep', 'num__full_sleep_breathing_rate', 'num__daily_temperature_variation', 'num__minutes_in_default_zone_1', 'num__wake_after_sleep_pct', 'num__calories', 'num__active_to_rest_transition', 'num__rmssd', 'num__sleep_activity_balance', 'num__deep_sleep_score', 'num__steps_norm_cal', 'num__sleep_wake_ratio', 'num__sleep_rem_ratio', 'num__resting_hr', 'num__minutes_in_default_zone_2', 'num__steps', 'num__sedentary_minutes', 'num__sleep_light_ratio']

Entrenament models amb Perm_Top3
==== LGBM ====
Entrenant model...
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive 

In [24]:
display(perm_results_df[display_cols])

,Experiment,Train F1 (1),Train F1 (macro global),Train Accuracy,Test Recall (1),Test Precision (1),Test F1 (1),Test F1 (macro global),Test Accuracy
0,LGBM_Perm_Top3,0.61463,0.56258,0.56878,0.77841,0.42947,0.55354,0.51430,0.51747
1,LGBM_Perm_Top4,0.62014,0.58027,0.58406,0.76136,0.42949,0.54918,0.51758,0.51965
2,LGBM_Perm_Top5,0.63696,0.60623,0.60862,0.78409,0.44089,0.56442,0.53279,0.53493
3,LGBM_Perm_Top6,0.67842,0.67124,0.67140,0.70455,0.43662,0.53913,0.53711,0.53712
4,LGBM_Perm_Top7,0.69026,0.68941,0.68941,0.67614,0.45247,0.54214,0.56038,0.56114
5,LGBM_Perm_Top8,0.70368,0.70577,0.70579,0.66477,0.42238,0.51656,0.52178,0.52183
6,LGBM_Perm_Top9,0.70909,0.72009,0.72052,0.67614,0.44906,0.53968,0.55616,0.55677
7,LGBM_Perm_Top10,0.71109,0.71816,0.71834,0.72159,0.45848,0.56071,0.56545,0.56550
8,LGBM_Perm_Top11,0.72137,0.73261,0.73308,0.72727,0.48120,0.57919,0.59339,0.59389
9,LGBM_Perm_Top12,0.72439,0.73384,0.73417,0.76136,0.48201,0.59031,0.59386,0.59389


In [25]:
update_experiments_file(perm_results_df)


Métriques guardades a ../results/02_experiments/experiments.csv



## EXPERIMENT 2: PCA


Es realitza una anàlisi de components principals (PCA) per examinar com evolucionen els components més rellevants del conjunt de dades en termes de variància explicada acumulada, considerant els primers 3, 4, 5, 6, 7, 8, 9, 10 components

In [26]:
pca = PCA(random_state=42)
pca.fit(X_train_transformed)

explained_cumsum = pca.explained_variance_ratio_.cumsum()*100

ks = [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
cums = explained_cumsum[[k-1 for k in ks]]

plt.figure(figsize=(8, 4))
plt.plot(
    range(1, len(explained_cumsum) + 1),
    explained_cumsum,
)
plt.scatter(ks, cums)
plt.xlabel('Número de componentes')
plt.ylabel('Varianza explicada acumulada')
plt.title('Evolución de la varianza explicada según n_components')
plt.grid(True)
plt.tight_layout()
plt.show()

In [27]:
n_components_list = [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
for k in n_components_list:
    # Ajusta PCA
    pca = PCA(n_components=k, random_state=42)
    pca.fit(X_train_transformed)

    # loadings: matriz (n_features, k)
    loadings = pca.components_.T

    # importancia = suma de cargas absolutes de cada feature en tots els components
    importance = np.sum(np.abs(loadings), axis=1)

    # crea DataFrame, ordena top-k
    df_imp = pd.DataFrame({
        'feature':    feature_names,
        'importance': importance
    }).sort_values('importance', ascending=False).reset_index(drop=True)
    topk = df_imp.head(k)

    plt.figure()
    plt.barh(topk['feature'][::-1], topk['importance'][::-1])
    plt.xlabel('Importancia (suma de |carregues|)')
    plt.title(f'Top {k} features segons PCA')
    plt.tight_layout()
    plt.show()

In [28]:
# Guarda resultats i models
pca_results = []

for k in n_components_list:
    print(f"\n-- PCA - {k} components --")

    for model, classifier in CLASSIFIERS_FILTER.items():
        if model == "BalancedRandomForest":
            pipeline = ImbPipeline([
                ("preprocessor", preprocessor),
                ("pca",          PCA(n_components=k, random_state=42)),
                ("classifier",   classifier)
            ])
        else:
            pipeline = ImbPipeline([
                ("preprocessor", preprocessor),
                ("balance",       balance_method),
                ("pca",          PCA(n_components=k, random_state=42)),
                ("classifier",   classifier)
            ])

        best_est, y_train_pred, train_report, y_test_pred, test_report, best_params, best_score = train_models( 
        X_train, 
        y_train,
        X_test,
        y_test,
        pipeline, 
        PARAM_GRIDS[model],
        search_type="grid"
        )

        df_pca_results = append_results(
            pca_results,
            model,
            train_report,
            test_report,
            best_params,
            best_score,
            experiment=f"PCA_{k}",
        )



-- PCA - 5 components --
Entrenant model...
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

In [29]:
display(df_pca_results[display_cols])
update_experiments_file(df_pca_results)

,Experiment,Train F1 (1),Train F1 (macro global),Train Accuracy,Test Recall (1),Test Precision (1),Test F1 (1),Test F1 (macro global),Test Accuracy
0,LGBM_PCA_5,0.67314,0.64899,0.65066,0.71591,0.39748,0.51116,0.47071,0.47380
1,LGBM_PCA_6,0.68480,0.66350,0.66485,0.78409,0.41071,0.53906,0.47745,0.48472
2,LGBM_PCA_7,0.70074,0.68955,0.68996,0.77841,0.40774,0.53516,0.47302,0.48035
3,LGBM_PCA_8,0.71166,0.70848,0.70852,0.76705,0.41411,0.53785,0.48873,0.49345
4,LGBM_PCA_9,0.71674,0.71170,0.71179,0.76705,0.42056,0.54326,0.50075,0.50437
5,LGBM_PCA_10,0.72276,0.71939,0.71943,0.77841,0.43082,0.55466,0.51666,0.51965
6,LGBM_PCA_11,0.72243,0.72106,0.72107,0.75568,0.42222,0.54175,0.50617,0.50873
7,LGBM_PCA_12,0.74140,0.74556,0.74563,0.76705,0.43548,0.55556,0.52661,0.52838
8,LGBM_PCA_13,0.73551,0.73850,0.73854,0.77841,0.42679,0.55131,0.50954,0.51310
9,LGBM_PCA_14,0.76420,0.77512,0.77566,0.76136,0.43506,0.55372,0.52686,0.52838



Métriques guardades a ../results/02_experiments/experiments.csv



## Anàlisi de resultats

In [30]:
# Llegeix la taula d'experiments (ja ordenada per F1)
experiments_df = pd.read_csv("../results/02_experiments/experiments.csv")
top5 = experiments_df.head(5)

# Mostra els top 5
display(top5)


,Experiment,Train F1 (1),Train F1 (macro global),Train Accuracy,Test Precision (1),Test Recall (1),Test F1 (1),Test F1 (macro global),Test Accuracy,Best Params
0,LGBM_Perm_Top13,0.74087,0.75171,0.75218,0.50558,0.77273,0.61124,0.62197,0.62227,"{'classifier__subsample': 0.75, 'classifier__s..."
1,LGBM_FI_Top6,0.67466,0.67413,0.67413,0.48754,0.77841,0.59956,0.60043,0.60044,NaN
2,LGBM_Perm_Top12,0.72439,0.73384,0.73417,0.48201,0.76136,0.59031,0.59386,0.59389,"{'classifier__subsample': 0.75, 'classifier__s..."
3,LGBM_Perm_Top11,0.72137,0.73261,0.73308,0.48120,0.72727,0.57919,0.59339,0.59389,"{'classifier__subsample': 0.75, 'classifier__s..."
4,LGBM_Perm_Top15,0.74177,0.75526,0.75600,0.47253,0.73295,0.57461,0.58281,0.58297,"{'classifier__subsample': 0.75, 'classifier__s..."
